In [15]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('news_sentiment.csv')

df.head()

,sentence,updated_sentiment,sentiment_numeric,date
0,Model Suggests BTC Price Floor Is $39K Survey ...,positive,1,2021-09-16T23:30:14+00:00
1,Microstrategy Buys 5050 More Bitcoins Now Hodl...,positive,1,2021-09-13T12:30:07+00:00
2,Salvadoran Governments Chivo Wallet Experience...,negative,-1,2021-09-08T15:30:16+00:00
3,First Day of Bitcoin as Legal Tender: El Salva...,positive,1,2021-09-07T21:30:27+00:00
4,Matrixport Launches BTC-U Range Sniper Returns...,positive,1,2021-09-07T09:30:49+00:00


In [3]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer

tokenizer = RegexpTokenizer(r'\w+')

token_list = []
for sentence in df['sentence']:
    
        tokens = tokenizer.tokenize(sentence)
        lower_list = []
        for token in tokens:
            result = ''.join(i for i in token if not i.isdigit())
            result = PorterStemmer().stem(result)
            if result.lower() == 'btc':
                result = 'bitcoin'
            if len(result) > 1:
                lower_list.append(result.lower())
            
        token_list.append(lower_list)

In [4]:
sentences_processed = []
for token in token_list:
    str1 = " "  
    sentences_processed.append(str1.join(token))
    

In [5]:
X = sentences_processed
y = df['updated_sentiment']

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [7]:
import nltk
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words=stopwords, min_df=5, ngram_range=(1,2))

vectorizer.fit(X)

X_test_vector = vectorizer.transform(X_test)
X_train_vector = vectorizer.transform(X_train)


In [9]:
from joblib import dump

dump(vectorizer, 'tfid_vectorizer.joblib')

['tfid_vectorizer.joblib']

In [9]:
from sklearn.preprocessing import LabelEncoder

# label-encode data

label_encoder = LabelEncoder()
label_encoder.fit(y)

y_test_encoded = label_encoder.transform(y_test)
y_train_encoded = label_encoder.transform(y_train)


In [10]:
from sklearn.svm import SVC 
model = SVC(kernel='linear', C=1, gamma='auto')
model.fit(X_train_vector, y_train_encoded)

# param_grid = {'C': [1,5,10,25,50,100],
#               'gamma': ['auto', .00001, .0001, .001, .005, .01, .05, .1, .5, 1]}


# from sklearn.ensemble import RandomForestClassifier
# model = RandomForestClassifier(n_estimators=200)
# model.fit(X_train_vector, y_train_encoded)

# from xgboost import XGBClassifier

# model = XGBClassifier(max_depth=8, n_estimators=500)
# model.fit(X_train_vector, y_train_encoded)

# param_grid = {'n_estimators': [100, 200, 500],
#               'max_depth': [4, 8, 10]
#               'gamma': np.arange(0:1,.01)}



SVC(C=1, gamma='auto', kernel='linear')

In [11]:
# from sklearn.model_selection import GridSearchCV

# grid = GridSearchCV(model, param_grid, verbose=3)

In [12]:
# grid.fit(X_train_vector, y_train_encoded)

In [13]:
# # List the best parameters for this dataset
# print(grid.best_params_)

In [14]:
# # List the best score
# print(grid.best_score_)

In [15]:
predictions = model.predict(X_test_vector)

In [16]:
model.score(X_test_vector, y_test_encoded)

0.6511156186612576

In [17]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(y_test_encoded, predictions,
                            target_names=["negative", "neutral", "positive"]))

              precision    recall  f1-score   support

    negative       0.67      0.45      0.54       221
     neutral       0.64      0.84      0.73       502
    positive       0.67      0.46      0.55       263

    accuracy                           0.65       986
   macro avg       0.66      0.58      0.60       986
weighted avg       0.66      0.65      0.64       986



In [18]:
from joblib import dump

X_vector = vectorizer.transform(X)
y_encoded = label_encoder.transform(y)

model = SVC(kernel='linear', C=1, gamma='auto')
model.fit(X_vector, y_encoded)

dump(model, 'headline_classifier.joblib')

['headline_classifier.joblib']

In [19]:
# from numpy.random import seed
# seed(42)
# from tensorflow import random
# random.set_seed(42)

In [20]:
# from tensorflow.keras.utils import to_categorical

# # one-hot encode

# y_test_onehot = to_categorical(y_test_encoded)
# y_train_onehot = to_categorical(y_train_encoded)

In [21]:
# # first, create a normal neural network with 2 inputs, 6 hidden nodes, and 2 outputs
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense

# deep_model = Sequential()
# deep_model.add(Dense(units=6, activation='relu', input_dim=2))
# deep_model.add(Dense(units=2, activation='softmax'))

In [22]:
# deep_model.summary()

In [23]:
# # Compile the model
# deep_model.compile(optimizer='adam',
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

In [24]:
# import tensorflow as tf

# # X_test_vector = tf.keras.layers.TextVectorization(X_test, max_tokens=None, output_mode='int')
# # X_train_vector = tf.keras.layers.TextVectorization(X_train)

# vectorize_layer = tf.keras.layers.TextVectorization(
#  max_tokens=None,
#  output_mode='tf_idf')

# # Now that the vocab layer has been created, call `adapt` on the text-only
# # dataset to create the vocabulary. You don't have to batch, but for large
# # datasets this means we're not keeping spare copies of the dataset.
# vectorize_layer.adapt(X)

# # Create the model that uses the vectorize text layer
# deep_model = tf.keras.models.Sequential()

# # Start by creating an explicit input layer. It needs to have a shape of
# # (1,) (because we need to guarantee that there is exactly one string
# # input per batch), and the dtype needs to be 'string'.
# deep_model.add(tf.keras.Input(shape=(1,), dtype=tf.string))

# # The first layer in our model is the vectorization layer. After this
# # layer, we have a tensor of shape (batch_size, max_len) containing vocab
# # indices.
# deep_model.add(vectorize_layer)

# # Now, the model can map strings to integers, and you can add an embedding
# # layer to map these integers to learned embeddings.
# # input_data = [["foo qux bar"], ["qux baz"]]
# # model.predict(input_data)



In [25]:
# # Fit the model to the training data
# deep_model.fit(
#     X_train.to_numpy(),
#     y_train_onehot,
#     epochs=100,
#     shuffle=True,
#     verbose=2
# )